# Demo 06 - Merge, Join, and SQL

In this demo we will first see how to do merges and joins in Pandas and then try them with an instance of SQLite

In [ ]:
# Only run in COLAB!
# clone the course repository, change to right directory, and import libraries.
%cd /content
!git clone https://github.com/nmattei/cmps6790.git
%cd /content/cmps6790/_demos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
# Make the fonts a little bigger in our graphs.
font = {'size'   : 20}
plt.rc('font', **font)
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['pdf.fonttype'] = 42

## Joins in Pandas

Let's see a few [joins in Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) and how to execute them.

**Note:** We can also use the [merge command in pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) to joins as well.  We'll explore this more fully in the lab.  The key idea is that merge let's us pick columns to do the merge/join while join will always join on the index.

In [ ]:
# Careful here! We have to set the index!

df1 = pd.DataFrame([(1, 'foo', 3),
                    (2, 'bar', 2),
                    (3, 'foo', 4),
                    (4, 'foo', 3)],
                  columns=['ID','A', 'B'])
df1.set_index('ID', inplace=True)
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)],
                   columns=['ID','C'])
df2.set_index('ID', inplace=True)
display(df2)

In [ ]:
# Inner join
df1.join(df2, how='inner')

In [ ]:
# Can do inner, outer, left, right.
df1.join(df2, how='right')

If we don't have the same column names we can tell Pandas that we want to join on a specific column.

In [ ]:
df1 = pd.DataFrame([(1, 'foo', 3), (2, 'bar', 2), (3, 'foo', 4), (4, 'foo', 3)],
                  columns=['leftid','A', 'B'])
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)],
                   columns=['rightid','C'])
df2.set_index('rightid', inplace=True)
display(df2)

In [ ]:
# Inner join
df1.join(df2, how='inner')
# Is this what we wanted?!?!

In [ ]:
# We could have set the index but we can also tell Pandas to do it for us.
df1.join(df2, how='inner', on='leftid')

## Pandas Merge

Or we can skip all this foolishness by using the [pandas merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) command...

In [ ]:
df1 = pd.DataFrame([(1, 'foo', 3), (2, 'bar', 2), (3, 'foo', 4), (4, 'foo', 3)],
                  columns=['ID','A', 'B'])
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)],
                   columns=['ID','C'])
display(df2)

In [ ]:
df1.merge(df2, left_on="ID", right_on="ID", how='inner')

In [ ]:
df1.merge(df2, left_on="ID", right_on="ID", how='outer')

In [ ]:
df1.merge(df2, left_on="ID", right_on="ID", how='left')

In [ ]:
df1.merge(df2, left_on="ID", right_on="ID", how='right')

## Using SQL with SQLite!

Helpful here is the [SQLite Cheat Sheet](https://www.sqlitetutorial.net/sqlite-cheat-sheet/) and [Python library](https://docs.python.org/3/library/sqlite3.html) documentation.

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
!pwd

In [ ]:
# Create a DB and connect to it.
# These creates a new file to store the database.
conn = sqlite3.connect("lec09-n.db")
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE cats (
    cat_id INTEGER PRIMARY KEY,
    name TEXT
)''')


In [ ]:
# Insert into the table
cursor.execute('''INSERT INTO cats VALUES (1, "Megabyte")''')
cursor.execute('''INSERT INTO cats VALUES (2, "Meowly Cyrus")''')
cursor.execute('''INSERT INTO cats VALUES (3, "Fuzz Aldrin")''')
cursor.execute('''INSERT INTO cats VALUES (4, "Chairman Meow")''')
cursor.execute('''INSERT INTO cats VALUES (5, "Anderson Pooper")''')
cursor.execute('''INSERT INTO cats VALUES (6, "Gigabyte")''')
# Commit will update the database on disk with a set of changes.
conn.commit()


In [ ]:
# What if we run an SQL Command?
cursor.execute('''SELECT * FROM cats''')

In [ ]:
# We need to iterate over the object we get back...
for row in cursor.execute('''SELECT * FROM cats'''):
    print(row)

In [ ]:
# Or we can just go to Pandas from here...
# note that we can tell Pandas what to do for an index here.
pd.read_sql_query("SELECT * FROM cats", conn, index_col="cat_id")

In [ ]:
# This is a full up DB so we can delete stuff too..
cursor.execute('''DELETE FROM cats WHERE cat_id == 4''')
conn.commit()
pd.read_sql_query("SELECT * FROM cats", conn, index_col="cat_id")

In [ ]:
# Let's make another table for joins.
cursor.execute('''
CREATE TABLE visits (
    visit_id INTEGER PRIMARY KEY,
    date DATE,
    cat_id INTEGER,
    FOREIGN KEY(cat_id) REFERENCES cats(cat_id)
)''')

In [ ]:
cursor.execute('''INSERT INTO visits VALUES (1, "02-16-2017", 1)''')
cursor.execute('''INSERT INTO visits VALUES (2, "02-14-2017", 1)''')
cursor.execute('''INSERT INTO visits VALUES (5, "02-03-2017", 2)''')
conn.commit()

In [ ]:
# Check what we did...
pd.read_sql_query("SELECT * FROM visits", conn, index_col="visit_id")

# Joins in SQL

Note that the SELECT command can give us an inner join by default and we don't have to use the keyword INNER JOIN.

In [ ]:
# We can do our join in Pandas
df_cats = pd.read_sql_query('''SELECT * from cats''', conn)
display(df_cats)
df_visits = pd.read_sql_query('''SELECT * from visits''', conn)
display(df_visits)

In [ ]:
# Here we bring down all the data then do our join..
df_cats.merge(df_visits, how = "inner",
              left_on = "cat_id", right_on = "cat_id")

In [ ]:
# Or we can directly run them on the system

result = cursor.execute('''
               SELECT
                   *
               FROM
                   cats, visits
               WHERE
                   cats.cat_id == visits.cat_id
               ''')
for i in result:
    print(i)


In [ ]:
# We can also use union to combine rows, even if it doesn't make sense.
result = cursor.execute('''
               SELECT
                   *
               FROM
                   cats
               UNION
               SELECT
                   cat_id, date
               FROM
                   visits
               ''')
for i in result:
    print(i)

In [ ]:
# What if we want a different join?
df_cats.merge(df_visits, how = "left",
              left_on = "cat_id", right_on = "cat_id")



In [ ]:
result = cursor.execute('''SELECT * FROM
                                cats
                           LEFT JOIN
                               visits
                           ON
                               cats.cat_id == visits.cat_id''')
for i in result:
    print(i)

In [ ]:
# What if we want a different join?
df_cats.merge(df_visits, how = "outer",
              left_on = "cat_id", right_on = "cat_id")



In [ ]:
# Let's do an Outer Join!

result = cursor.execute('''SELECT * FROM cats FULL OUTER JOIN visits
                    ON cats.cat_id == visits.cat_id''')
for i in result:
    print(i)

So we have to be careful as SQLite has some limitations... but you get the idea.


## GroupBy and Aggregate in SQL.


In [ ]:
# Make a new Table...
cursor.execute('''
CREATE TABLE persons (
    id INTEGER PRIMARY KEY,
    age FLOAT,
    wgt_kg FLOAT,
    hgt_cm FLOAT,
    nat_id INTEGER

)''')

In [ ]:
# Insert into the table
cursor.execute('''INSERT INTO persons VALUES (1, 12.2, 42.3, 145.1, 1)''')
cursor.execute('''INSERT INTO persons VALUES (2, 11.0, 40.8, 143.8, 1)''')
cursor.execute('''INSERT INTO persons VALUES (3, 15.6, 65.3, 165.3, 2)''')
cursor.execute('''INSERT INTO persons VALUES (4, 35.1, 84.2, 185.8, 1)''')
cursor.execute('''INSERT INTO persons VALUES (5, 18.1, 62.2, 176.2, 3)''')
cursor.execute('''INSERT INTO persons VALUES (6, 19.6, 82.1, 180.1, 1)''')
conn.commit()

In [ ]:
ret = cursor.execute('''SELECT * FROM persons''')
for row in ret:
    print(row)

In [ ]:
ret = cursor.execute('''SELECT nat_id, wgt_kg, AVG(age) as average_age
                        FROM persons
                        GROUP BY nat_id''')
for row in ret:
    print(row)

### Make sure you close your connection or things go badly!!

- Writes all the changes to the database.

In [ ]:
conn.close()

### More practice in SQL

That about does it for the SQL for this class -- You can do the [SQL murder mystery!](http://mystery.knightlab.com/walkthrough.html)